In [1]:
# Importing Libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from itertools import chain
from collections import Counter
import operator as op
# Adjustment Options
pd.set_option('display.max_columns', None) # Display All Columns given the dataset has many columns
pd.set_option('display.max_rows',75)


# Exploring .csv kaggle files
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.split(".")[-1] == "csv":
            print(os.path.join(dirname, filename))


/kaggle/input/mental-health-in-tech-2016/mental-heath-in-tech-2016_20161114.csv


In [2]:
df = pd.read_csv("/kaggle/input/mental-health-in-tech-2016/mental-heath-in-tech-2016_20161114.csv")
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,Do you have medical coverage (private insurance or state-provided) which includes treatment of mental health issues?,Do you know local or online resources to seek help for a mental health disorder?,"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?","If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?","If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?","If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?",Do you believe your productivity is ever affected by a mental health issue?,"If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health issues and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Do you think that discussing a physical health issue with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your previous co-workers?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you be willing to bring up a physical health issue with a potential employer in an interview?,Why or why not?,Would you bring up a mental health issue with a potential employer in an interview?,Why or why not?.1,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a menta

In [3]:
for i, column in enumerate(df.columns):
    print(i, column)

0 Are you self-employed?
1 How many employees does your company or organization have?
2 Is your employer primarily a tech company/organization?
3 Is your primary role within your company related to tech/IT?
4 Does your employer provide mental health benefits as part of healthcare coverage?
5 Do you know the options for mental health care available under your employer-provided coverage?
6 Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?
7 Does your employer offer resources to learn more about mental health concerns and options for seeking help?
8 Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?
9 If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:
10 Do you think that discussing a mental health disorder with your employer would have negative consequences?

# Data Cleaning
- Dealing with Nan values
- Dealing with ordinal and nominal columns
- Normalizing Data

In [4]:
# a threshold for Nan values
nan_column_threshold = df.shape[0]*.25
nan_row_threshold = df.shape[1]*.25
nan_column_threshold, nan_row_threshold

(358.25, 15.75)

In [5]:
df.isna().sum()

Are you self-employed?                                                                                                                                                                 0
How many employees does your company or organization have?                                                                                                                           287
Is your employer primarily a tech company/organization?                                                                                                                              287
Is your primary role within your company related to tech/IT?                                                                                                                        1170
Does your employer provide mental health benefits as part of healthcare coverage?                                                                                                    287
Do you know the options for mental health care available under your employe

In [6]:
df.loc[:,df.isna().sum() > nan_column_threshold]

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Is your primary role within your company related to tech/IT?,Do you know the options for mental health care available under your employer-provided coverage?,Do you have medical coverage (private insurance or state-provided) which includes treatment of mental health issues?,Do you know local or online resources to seek help for a mental health disorder?,"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?","If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?","If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?","If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?",Do you believe your productivity is ever affected by a mental health issue?,"If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",Have your observations of how another individual who discussed a mental health disorder made you less likely to reveal a mental health issue yourself in your current workplace?,"If yes, what condition(s) have you been diagnosed with?","If maybe, what condition(s) do you believe you have?","If so, what condition(s) were you diagnosed with?",What US state or territory do you live in?,What US state or territory do you work in?
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,NaN
1,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,"Anxiety Disorder (Generalized, Social, Phobia,...",Illinois,Illinois
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,"Yes, I know several","Sometimes, if it comes up",I'm not sure,"Sometimes, if it comes up",I'm not sure,Yes,1-25%,NaN,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,NaN
4,1.0,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,"Anxiety Disorder (Generalized, Social, Phobia,...",Illinois,Illinois
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,NaN,NaN,1.0,"Yes, I know several",Not applicable to me,NaN,Not applicable to me,Not applicable to me,Not applicable to me,NaN,NaN,NaN,NaN,NaN,New York,New York
1429,NaN,NaN,1.0,I know some,"No, because it doesn't matter",No,"No, because it doesn't matter",No,No,NaN,Maybe,NaN,NaN,"Psychotic Disorder (Schizophrenia, Schizoaffec...",California,NaN
1430,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,"Anxiety Disorder (Generalized, Social, Phobia,...","Anxiety Disorder (Generalized, Social, Phobia,...",Georgia,Georgia
1431,1.0,I am not sure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maybe,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,"Mood Disorder (Depression, Bipolar Disorder, etc)",Nebraska,Nebraska


In [7]:
df.loc[df.isna().sum(axis=1) > nan_row_threshold,:]

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,Do you have medical coverage (private insurance or state-provided) which includes treatment of mental health issues?,Do you know local or online resources to seek help for a mental health disorder?,"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?","If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?","If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?","If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?",Do you believe your productivity is ever affected by a mental health issue?,"If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health issues and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Do you think that discussing a physical health issue with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your previous co-workers?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you be willing to bring up a physical health issue with a potential employer in an interview?,Why or why not?,Would you bring up a mental health issue with a potential employer in an interview?,Why or why not?.1,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a menta

In [8]:
df.loc[df.isna().sum(axis=1) > nan_row_threshold,:].isna().sum(axis=1)

0       17
3       19
9       18
17      16
18      28
        ..
1420    21
1423    26
1428    21
1429    30
1432    26
Length: 494, dtype: int64

## Nan Values
- 16 columns have more than 25% of Nan values but this could be to no answer or inapplicable. to be reviewed.
- 494 respondents didnt answer more than 25% of the survey. Theyll be dropped and considered inaccurate responses. 

In [9]:
df_2 = df.loc[df.isna().sum(axis=1) < nan_row_threshold,:]
df_2.shape

(939, 63)

In [10]:
df_2.loc[:,df_2.isna().sum() > nan_column_threshold] # Checking again for column Nans

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Is your primary role within your company related to tech/IT?,Do you have medical coverage (private insurance or state-provided) which includes treatment of mental health issues?,Do you know local or online resources to seek help for a mental health disorder?,"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?","If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?","If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?","If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?",Do you believe your productivity is ever affected by a mental health issue?,"If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",Have your observations of how another individual who discussed a mental health disorder made you less likely to reveal a mental health issue yourself in your current workplace?,"If yes, what condition(s) have you been diagnosed with?","If maybe, what condition(s) do you believe you have?","If so, what condition(s) were you diagnosed with?"
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,"Anxiety Disorder (Generalized, Social, Phobia,..."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,"Anxiety Disorder (Generalized, Social, Phobia,..."
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mood Disorder (Depression, Bipolar Disorder, etc)",NaN
1426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Maybe,Attention Deficit Hyperactivity Disorder,NaN,Attention Deficit Hyperactivity Disorder
1427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,"Anxiety Disorder (Generalized, Social, Phobia,..."
1430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,"Anxiety Disorder (Generalized, Social, Phobia,...","Anxiety Disorder (Generalized, Social, Phobia,..."


In [11]:
df_2.loc[:,df_2.isna().sum() > nan_column_threshold].columns

Index(['Is your primary role within your company related to tech/IT?',
       'Do you have medical coverage (private insurance or state-provided) which includes treatment of  mental health issues?',
       'Do you know local or online resources to seek help for a mental health disorder?',
       'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?',
       'If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?',
       'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?',
       'If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?',
       'Do you believe your productivity is ever affected by a mental health issue?',
       'If yes, what percentage of your work time (time performing primary or secondary job func

## Fill NaN with inapplicable or no asnwer or drop

In [12]:
# No Answer Columns
# any columns with a determinate answer but with Nan will be dropped
drop_columns = ['Is your primary role within your company related to tech/IT?',
                'Do you have medical coverage (private insurance or state-provided) which includes treatment of \xa0mental health issues?',
                'Do you know local or online resources to seek help for a mental health disorder?'
               ]

# Questions that deal with belief or witness will be filled with 'no answer'
no_asnwers_cols = ['Do you believe your productivity is ever affected by a mental health issue?',
                  'Have your observations of how another individual who discussed a mental health disorder made you less likely to reveal a mental health issue yourself in your current workplace?'
                  ]

# Any 'If questions' will be filled with inapplicable 
inapplicable_cols = ['If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?',
                    'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?',
                    'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?',
                    'If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?',
                    'If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?',
                    'If yes, what condition(s) have you been diagnosed with?',
                    'If maybe, what condition(s) do you believe you have?',
                    'If so, what condition(s) were you diagnosed with?'
                    ]

In [13]:
df_3 = df_2.drop(columns=drop_columns, axis=1)
df_3

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?","If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?","If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?","If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?",Do you believe your productivity is ever affected by a mental health issue?,"If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health issues and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Do you think that discussing a physical health issue with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your previous co-workers?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you be willing to bring up a physical health issue with a potential employer in an interview?,Why or why not?,Would you bring up a mental health issue with a potential employer in an interview?,Why or why not?.1,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?,How willing would you be to share with friends and family that you have a mental illness?,Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?,Have your obse

In [14]:
df_3.loc[:,no_asnwers_cols] = df_3.loc[:,no_asnwers_cols].fillna("No Answer")

In [15]:
df_3.loc[:,inapplicable_cols] = df_3.loc[:,inapplicable_cols].fillna("Inapplicable")

In [16]:
df_3.loc[:,df_3.isna().sum() > 0]

,Do you know the options for mental health care available under your employer-provided coverage?,Why or why not?,Why or why not?.1,Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?,What is your gender?,What US state or territory do you live in?,What US state or territory do you work in?
1,Yes,It would depend on the health issue. If there ...,While mental health has become a more prominen...,No,male,Illinois,Illinois
2,NaN,"They would provable need to know, to Judge if ...","Stigma, mainly.",Maybe/Not sure,Male,NaN,NaN
4,Yes,Depending on the interview stage and whether I...,I don't know,"Yes, I experienced",Female,Illinois,Illinois
5,I am not sure,If it would potentially affect my ability to d...,It would depend on the field & what I knew of ...,"Yes, I experienced",Male,NaN,NaN
6,No,I want to gauge their ability to support this ...,"I want to gauge their ability to support, unde...",No,M,Tennessee,Tennessee
...,...,...,...,...,...,...,...
1425,I am not sure,NaN,Stigma.,No,Male,NaN,NaN
1426,I am not sure,NaN,NaN,"Yes, I experienced",Female,NaN,NaN
1427,No,I would be afraid it would reflect poorly on me.,I would be certain it would reflect poorly on me.,"Yes, I experienced",female,Illinois,Illinois
1430,Yes,Fear that doing so would cause the employer to...,Fear that the employer would consider addition...,"Yes, I observed",Male,Georgia,Georgia


In [17]:
for column in df_3.loc[:,df_3.isna().sum() > 0].columns:
    print(f"{column}:\t{df_3[column].isna().sum()}")

Do you know the options for mental health care available under your employer-provided coverage?:	85
Why or why not?:	189
Why or why not?.1:	164
Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?:	28
What is your gender?:	3
What US state or territory do you live in?:	306
What US state or territory do you work in?:	302


In [18]:
df_4 = df_3.copy()

In [19]:
df_4.loc[:,df_4.isna().sum() > 0] = df_4.loc[:,df_4.isna().sum() > 0].fillna("No Answer")

In [20]:
df_4.loc[:,df_4.isna().sum() > 0]

""
1
2
4
5
6
...
1425
1426
1427
1430


All Nan Values have been adjusted

In [21]:
df_4.reset_index(drop=True,inplace=True)

## Noramlizing Answers in the Dataset

In [22]:
df_4.head()

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?","If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?","If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?","If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?",Do you believe your productivity is ever affected by a mental health issue?,"If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health issues and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Do you think that discussing a physical health issue with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your previous co-workers?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you be willing to bring up a physical health issue with a potential employer in an interview?,Why or why not?,Would you bring up a mental health issue with a potential employer in an interview?,Why or why not?.1,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?,How willing would you be to share with friends and family that you have a mental illness?,Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?,Have your obse

In [23]:
one_unique_answer = []
for i,column in enumerate(df_4.columns):
    if len(df_4[column].unique()) > 1:
        print(f"****{i, column}****")
        print(df_4[column].unique())
        print("\n")
    else:
        one_unique_answer.append(column)
print("The following columns have one unique answer:")
print(*one_unique_answer,sep="\n")
    

****(1, 'How many employees does your company or organization have?')****
['6-25' 'More than 1000' '26-100' '100-500' '1-5' '500-1000']


****(2, 'Is your employer primarily a tech company/organization?')****
[1. 0.]


****(3, 'Does your employer provide mental health benefits as part of healthcare coverage?')****
['No' 'Yes' "I don't know" 'Not eligible for coverage / N/A']


****(4, 'Do you know the options for mental health care available under your employer-provided coverage?')****
['Yes' 'No Answer' 'I am not sure' 'No']


****(5, 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?')****
['Yes' 'No' "I don't know"]


****(6, 'Does your employer offer resources to learn more about mental health concerns and options for seeking help?')****
['Yes' 'No' "I don't know"]


****(7, 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided

## First Iteration of Normalizing and Further cleaning
- Columns with one unique answers will be dropped.
- Why or Why not columns need sentiment analysis but for this exercise, will be dropped.
- US terrirotires question will be dropped.
- Genders will be condensed to Male, Female and Other
- questions 44-46 that will be dropped as 47 is representative

In [24]:
df_5 = df_4.drop(one_unique_answer,axis=1)

In [25]:
dropped_columns = ['What US state or territory do you work in?',
                  'What US state or territory do you live in?',
                  'Why or why not?.1',
                  'Why or why not?',
                  'Do you currently have a mental health disorder?',
                  'If yes, what condition(s) have you been diagnosed with?',
                  'If maybe, what condition(s) do you believe you have?']

df_5 = df_5.drop(dropped_columns, axis=1)

In [26]:
def normalize_gender(gender):
    if type(gender) != str:
        raise ValueError("Gender must be a string")
    gender = gender.strip().lower()
    for female in ['female', 'woman','f']:   
        if op.contains(gender,female):
            return 'Female'
    for male in ['male','man','m']:
        if op.contains(gender,male):
            return 'Male'
    return "Other"

In [27]:
df_5.loc[:,"What is your gender?"] = df_5.loc[:,"What is your gender?"].map(normalize_gender)

In [28]:
# doubel checking result
one_unique_answer = []
for i,column in enumerate(df_5.columns):
    if len(df_5[column].unique()) > 1:
        print(f"****{i, column}****")
        print(df_5[column].unique())
        print("\n")
    else:
        one_unique_answer.append(column)
print("The following columns have one unique answer:")
print(*one_unique_answer,sep="\n")

****(0, 'How many employees does your company or organization have?')****
['6-25' 'More than 1000' '26-100' '100-500' '1-5' '500-1000']


****(1, 'Is your employer primarily a tech company/organization?')****
[1. 0.]


****(2, 'Does your employer provide mental health benefits as part of healthcare coverage?')****
['No' 'Yes' "I don't know" 'Not eligible for coverage / N/A']


****(3, 'Do you know the options for mental health care available under your employer-provided coverage?')****
['Yes' 'No Answer' 'I am not sure' 'No']


****(4, 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?')****
['Yes' 'No' "I don't know"]


****(5, 'Does your employer offer resources to learn more about mental health concerns and options for seeking help?')****
['Yes' 'No' "I don't know"]


****(6, 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided

In [29]:
df_5["What country do you live in?"].value_counts()/df_5.shape[0]

What country do you live in?
United States of America    0.674121
United Kingdom              0.104366
Canada                      0.053248
Netherlands                 0.022364
Germany                     0.022364
Australia                   0.021299
Brazil                      0.009585
France                      0.008520
Ireland                     0.008520
Sweden                      0.008520
Switzerland                 0.005325
New Zealand                 0.005325
Finland                     0.004260
Bulgaria                    0.004260
Denmark                     0.004260
India                       0.003195
Spain                       0.003195
South Africa                0.003195
Chile                       0.003195
Russia                      0.003195
Pakistan                    0.003195
Norway                      0.002130
Afghanistan                 0.002130
Italy                       0.002130
Israel                      0.002130
Estonia                     0.002130
Colombia 

In [30]:
df_5['What country do you work in?'].value_counts()/df_5.shape[0]

What country do you work in?
United States of America    0.678381
United Kingdom              0.103301
Canada                      0.052183
Netherlands                 0.022364
Germany                     0.022364
Australia                   0.021299
Brazil                      0.009585
Sweden                      0.009585
France                      0.008520
Ireland                     0.008520
Switzerland                 0.005325
New Zealand                 0.005325
Bulgaria                    0.004260
Finland                     0.004260
Denmark                     0.004260
Russia                      0.003195
Chile                       0.003195
India                       0.003195
South Africa                0.003195
Estonia                     0.002130
Colombia                    0.002130
Spain                       0.002130
Norway                      0.002130
Pakistan                    0.002130
Israel                      0.002130
Austria                     0.001065
Bosnia an

In [31]:
df_5[df_5["What country do you work in?"].isin(["United States of America", "United Kingdom", "Canada", "Netherlands", "Germany", "Australia"])].shape

(845, 45)

In [32]:
df_5[df_5["What country do you live in?"].isin(["United States of America", "United Kingdom", "Canada", "Netherlands", "Germany", "Australia"])].shape

(843, 45)

In [33]:
remaining_obs = df_5[(df_5["What country do you live in?"].isin(["United States of America", "United Kingdom", "Canada", "Netherlands", "Germany", "Australia"]))&
(df_5["What country do you work in?"].isin(["United States of America", "United Kingdom", "Canada", "Netherlands", "Germany", "Australia"]))].shape[0]

total_obs = df_5.shape[0]

print(f"Drop percentage {1 - (remaining_obs/total_obs):.2f}")

Drop percentage 0.10


## Second Iteration of Normalizing and Further cleaning
- Create Dummy variables for questions 43 and 35
- Will only include United States of America, United Kingdom, Canada, Netherlands, Germany, Australia 

In [34]:
df_6 = df_5[(df_5["What country do you live in?"].isin(["United States of America", "United Kingdom", "Canada", "Netherlands", "Germany", "Australia"]))&
            (df_5["What country do you work in?"].isin(["United States of America", "United Kingdom", "Canada", "Netherlands", "Germany", "Australia"]))]
df_6.shape

(842, 45)

In [35]:
work_positions_df = df_6.iloc[:,43].str.get_dummies(sep="|")
work_positions_df.columns = [f"Work_Position_{x}" for x in work_positions_df.columns]
work_positions_df.head()

,Work_Position_Back-end Developer,Work_Position_Designer,Work_Position_Dev Evangelist/Advocate,Work_Position_DevOps/SysAdmin,Work_Position_Executive Leadership,Work_Position_Front-end Developer,Work_Position_HR,Work_Position_One-person shop,Work_Position_Other,Work_Position_Sales,Work_Position_Supervisor/Team Lead,Work_Position_Support
0,1,0,0,0,0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,1,1,1,1,0,0,0,0,1,1
3,1,1,0,1,0,1,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,0,0


In [36]:
df_7 = pd.concat([df_6,work_positions_df],axis=1).drop("Which of the following best describes your work position?",axis=1)

In [37]:
df_7.head()

,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health issues and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Do you think that discussing a physical health issue with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your previous co-workers?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you be willing to bring up a physical health issue with a potential employer in an interview?,Would you bring up a mental health issue with a potential employer in an interview?,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?,How willing would you be to share with friends and family that you have a mental illness?,Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?,Have your observations of how another individual who discussed a mental health disorder made you less likely to reveal a mental health issue yourself in your current workplace?,Do you have a family history of mental illness?,Have you had a mental health disorder in the past?,Have you been diagnosed with a mental health condition by a medical professional?,"If so, what condition(s) were you diagnosed with?",Have you ever sought treatment for a mental health issue from a mental health professional?,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What c

In [38]:
mental_health_df = df_7.iloc[:,35].str.get_dummies(sep="|")
mental_health_df.columns = [f"{x}_Diagnosed_Mental_Health" for x in mental_health_df.columns]
mental_health_df.head()

,ADD (w/o Hyperactivity)_Diagnosed_Mental_Health,Addictive Disorder_Diagnosed_Mental_Health,"Anxiety Disorder (Generalized, Social, Phobia, etc)_Diagnosed_Mental_Health",Asperger Syndrome_Diagnosed_Mental_Health,Attention Deficit Disorder_Diagnosed_Mental_Health,Attention Deficit Hyperactivity Disorder_Diagnosed_Mental_Health,Autism_Diagnosed_Mental_Health,Autism (Asperger's)_Diagnosed_Mental_Health,"Autism - while not a ""mental illness"", still greatly affects how I handle anxiety_Diagnosed_Mental_Health",Autism Spectrum Disorder_Diagnosed_Mental_Health,Dissociative Disorder_Diagnosed_Mental_Health,"Eating Disorder (Anorexia, Bulimia, etc)_Diagnosed_Mental_Health",Gender Identity Disorder_Diagnosed_Mental_Health,Inapplicable_Diagnosed_Mental_Health,"Mood Disorder (Depression, Bipolar Disorder, etc)_Diagnosed_Mental_Health",Obsessive-Compulsive Disorder_Diagnosed_Mental_Health,"Personality Disorder (Borderline, Antisocial, Paranoid, etc)_Diagnosed_Mental_Health",Post-traumatic Stress Disorder_Diagnosed_Mental_Health,"Psychotic Disorder (Schizophrenia, Schizoaffective, etc)_Diagnosed_Mental_Health",Seasonal Affective Disorder_Diagnosed_Mental_Health,Stress Response Syndromes_Diagnosed_Mental_Health,Substance Use Disorder_Diagnosed_Mental_Health,Suicidal Ideation_Diagnosed_Mental_Health,attention deficit disorder (but not the hyperactive version)_Diagnosed_Mental_Health,posttraumatic stress disourder_Diagnosed_Mental_Health
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [39]:
df_8 = pd.concat([df_7,mental_health_df],axis=1).drop(['Have you been diagnosed with a mental health condition by a medical professional?',
                                                       'If so, what condition(s) were you diagnosed with?'],axis=1)

In [40]:
# doubel checking result
for i,column in enumerate(df_8.columns):
    print(f"****{i, column}****")
    print(df_8[column].unique())
    print("\n")


****(0, 'How many employees does your company or organization have?')****
['6-25' 'More than 1000' '26-100' '100-500' '1-5' '500-1000']


****(1, 'Is your employer primarily a tech company/organization?')****
[1. 0.]


****(2, 'Does your employer provide mental health benefits as part of healthcare coverage?')****
['No' 'Yes' "I don't know" 'Not eligible for coverage / N/A']


****(3, 'Do you know the options for mental health care available under your employer-provided coverage?')****
['Yes' 'No Answer' 'I am not sure' 'No']


****(4, 'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?')****
['Yes' 'No' "I don't know"]


****(5, 'Does your employer offer resources to learn more about mental health concerns and options for seeking help?')****
['Yes' 'No' "I don't know"]


****(6, 'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided

In [41]:
df_8.shape

(842, 79)

In [42]:
# mhc_types = df.iloc[:,48].apply(lambda x:x.split("|")).to_list()

In [43]:
# mhc_types

In [44]:
# mhc_types = list(chain.from_iterable(mhc_types))

In [45]:
# mhc_types

In [46]:
# mhc_db = dict(Counter(mhc_types))

In [47]:
# other = 0
# new_mhc_db = {}
# for key,value in mhc_db.items():
#     if value <= 15:
#         other += value
#     else:
#         new_mhc_db[key] = value    

In [48]:
# print(other)

In [49]:
# new_mhc_db["Other"] = other

In [50]:
# new_mhc_db

In [51]:
# plt.pie(new_mhc_db.values(), labels=new_mhc_db.keys(), autopct='%1.1f%%')
# plt.title('MHC Types')
# plt.show()

In [52]:
# gender_df = df.query("'Mood Disorder (Depression, Bipolar Disorder, etc)' in `If yes, what condition(s) have you been diagnosed with?`")

In [53]:
# gender_df["What is your gender?"].value_counts()

In [54]:
# plt.pie(gender_df["What is your gender?"].value_counts().values, labels=gender_df["What is your gender?"].value_counts().index, autopct='%1.1f%%')
# plt.title('Gender Segmentation of Mood Disorders')
# plt.show()

In [55]:
# df.iloc[:,9].unique()

In [56]:
# pd.get_dummies(df.iloc[:,9],prefix="Question 9")